<a href="https://colab.research.google.com/github/jliangtony/memo/blob/main/github_request_from_https_url.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GitHubのアクセストークンを取得する手順を説明します。
アクセストークンの発行手順
1. GitHubにログイン
GitHubウェブサイトにログインします
画面右上のプロフィールアイコンをクリック
2. 設定画面へ移動
Settings → Developer settings を選択
左メニューから「Personal access tokens」を選択
「Generate new token (classic)」をクリック
3. トークン設定
Note: トークンの用途を記入
Expiration: トークンの有効期限を設定
Select scopes: 必要な権限を選択
リポジトリ操作の場合は「repo」にチェック
4. トークン生成
「Generate token」ボタンをクリック
表示されたトークンを必ずコピーして保存（この画面でしか表示されません）

コードでの使用方法

```
headers = {
    "Accept": "application/vnd.github+json",
    "Authorization": f"Bearer {YOUR_TOKEN}"  # コピーしたトークンを設定
}
```

In [ ]:
import requests
import base64
import re
from urllib.parse import urlparse

YOUR_TOKEN = "xxxxxxxxxxxxxxxxxxxxxxxxxx"

def parse_github_url(url):
    """GitHubのURLからowner名とリポジトリ名を抽出"""
    pattern = r"github\.com/([^/]+)/([^/]+)"
    match = re.search(pattern, url)
    if not match:
        raise ValueError("Invalid GitHub URL format")
    return match.group(1), match.group(2)

def is_github_url(url):
    """URLがGitHubのURLかどうかを確認"""
    parsed = urlparse(url)
    return parsed.netloc == "github.com"

def get_repo_contents(owner, repo, path=""):
    """リポジトリのコンテンツを取得"""
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{path}"
    headers = {
        "Accept": "application/vnd.github+json",
        "Authorization": f"Bearer {YOUR_TOKEN}"
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    return None

def get_file_content(owner, repo, path):
    """ファイルの内容を取得"""
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{path}"
    headers = {
        "Accept": "application/vnd.github+json",
        "Authorization": f"Bearer {YOUR_TOKEN}"
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        content = response.json()
        if content.get("encoding") == "base64":
            return base64.b64decode(content["content"]).decode()
    return None

def analyze_github_repo(github_url):
    """GitHubリポジトリを解析"""
    try:
        if not is_github_url(github_url):
            raise ValueError("This is not a GitHub URL")

        owner, repo = parse_github_url(github_url)
        contents = get_repo_contents(owner, repo)

        if contents:
            print(f"\nAnalyzing repository: {owner}/{repo}")
            print("\nRepository contents:")
            for item in contents:
                print(f"{'[DIR] ' if item['type'] == 'dir' else '[FILE]'} {item['path']}")
        else:
            print("No contents found or repository access denied")

    except ValueError as e:
        print(f"Error: {str(e)}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

# 使用例
github_url = "https://github.com/xxxxxxx/xxxxxxxx"
analyze_github_repo(github_url)
